### Calculate Accuracy for DPR outputs

In [3]:
import json
from operator import itemgetter
file_path = ## dpr results
K = 1

with open(file_path, 'r', encoding="utf8") as file :
    data = json.load(file)
    
correct_case = 0
for instance in data :
    ## sort by DPR score
    candidate_docs = instance['ctxs']
    candidate_docs = sorted(candidate_docs, key=itemgetter('score'), reverse=True)
    ## find answer document in top k passage
    answers = candidate_docs[:K]
    
    for answer in answers :
        if answer['has_answer'] :
            correct_case = correct_case + 1
            break

print("-- accuracy --")
print(correct_case)
print(len(data))
print(correct_case / len(data))

-- accuracy --
1309
3610
0.3626038781163435


### Merge output files for Multi DPR outputs

##### For low resource set up, introduce method which calculate similarty multiple wikipedia set and merge
##### All file path need have to be renewed (file path in this code is just examples)

In [38]:
import json
from operator import itemgetter

TOP_K=100

## initialize file
file_path = '/DPR/results/NQ_dev/NQ_dev_top100_0.json'
with open(file_path, 'r', encoding="utf8") as file :
        init_data = json.load(file)

final_file_path = "/DPR/results/NQ_dev/NQ_dev_top100_all.json"
with open(final_file_path, 'w', encoding='utf-8') as outfile:
    json.dump(init_data, outfile)
    
for file_num in range(1, 50):
    print("file number : " + str(file_num)) 
    file_path = '/DPR/results/NQ_dev/NQ_dev_top100_' + str(file_num) + '.json'

    with open(file_path, 'r', encoding="utf8") as file :
        data = json.load(file)
        
    with open(final_file_path, 'r', encoding="utf8") as file2 :
        data2 = json.load(file2)
        
        
    for i, instance_pair in enumerate(zip(data, data2)) :
        
        ## sort by DPR score
        candidate_docs1 = instance_pair[0]['ctxs']
        candidate_docs2 = instance_pair[1]['ctxs']
        candidate_docs = candidate_docs1 + candidate_docs2
        candidate_docs = sorted(candidate_docs, key=itemgetter('score'), reverse=True)
        
        ## find answer document in top k passage
        data2[i]["ctxs"] = candidate_docs[:100]
        
    with open(final_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(data2, outfile)

file number : 1
file number : 2
file number : 3
file number : 4
file number : 5
file number : 6
file number : 7
file number : 8
file number : 9
file number : 10
file number : 11
file number : 12
file number : 13
file number : 14
file number : 15
file number : 16
file number : 17
file number : 18
file number : 19
file number : 20
file number : 21
file number : 22
file number : 23
file number : 24
file number : 25
file number : 26
file number : 27
file number : 28
file number : 29
file number : 30
file number : 31
file number : 32
file number : 33
file number : 34
file number : 35
file number : 36
file number : 37
file number : 38
file number : 39
file number : 40
file number : 41
file number : 42
file number : 43
file number : 44
file number : 45
file number : 46
file number : 47
file number : 48
file number : 49


### make sh file for distributing calculate DPR score

In [18]:
template = """
CUDA_VISIBLE_DEVICES=6,7 \
HYDRA_FULL_ERROR=1 \
python dense_retriever.py \
model_file=/DPR/weights/downloads/checkpoint/retriever/single-adv-hn/nq/bert-base-encoder.cp \
qa_dataset=nq_test \
ctx_datatsets=[dpr_wiki] \
encoded_ctx_files=["/DPR/downloads/data/retriever_results/nq/single-adv-hn/wikipedia_passages_{i}.pkl"] \
out_file=/DPR/results/NQ_test/NQ_test_top100_{i}.json
"""

sh_file_path = "/DPR/calculate_score.sh"
file = open(sh_file_path, 'w', encoding="utf8")
for i in range(50) :
    file.write(template.format(i=i))
    print(template.format(i=i))
    #print('\n')
file.close()


CUDA_VISIBLE_DEVICES=6,7 HYDRA_FULL_ERROR=1 python dense_retriever.py model_file=/ssd0/byeongguk/DPR/weights/downloads/checkpoint/retriever/single-adv-hn/nq/bert-base-encoder.cp qa_dataset=nq_test ctx_datatsets=[dpr_wiki] encoded_ctx_files=["/ssd0/byeongguk/DPR/downloads/data/retriever_results/nq/single-adv-hn/wikipedia_passages_0.pkl"] out_file=/ssd0/byeongguk/DPR/results/NQ_test/NQ_test_top100_0.json


CUDA_VISIBLE_DEVICES=6,7 HYDRA_FULL_ERROR=1 python dense_retriever.py model_file=/ssd0/byeongguk/DPR/weights/downloads/checkpoint/retriever/single-adv-hn/nq/bert-base-encoder.cp qa_dataset=nq_test ctx_datatsets=[dpr_wiki] encoded_ctx_files=["/ssd0/byeongguk/DPR/downloads/data/retriever_results/nq/single-adv-hn/wikipedia_passages_1.pkl"] out_file=/ssd0/byeongguk/DPR/results/NQ_test/NQ_test_top100_1.json


CUDA_VISIBLE_DEVICES=6,7 HYDRA_FULL_ERROR=1 python dense_retriever.py model_file=/ssd0/byeongguk/DPR/weights/downloads/checkpoint/retriever/single-adv-hn/nq/bert-base-encoder.cp qa_da